In [2]:
import pandas as pd
import pickle
from src.utils.preprocess_data import preprocess_data

In [3]:
#Load the data
DATASET_PATH = "../../data/raw/nn_challenge_train.pkl"
df = pd.read_pickle(DATASET_PATH)

In [4]:
df_t_0 = df[df["target"] == 0]
df_t_1 = df[df["target"] != 0]
print(len(df_t_0))
print(len(df_t_1))

66466
405956


In [5]:
df_t_1_sub = df_t_1.sample(n=len(df_t_0)+30000, random_state=42)

In [6]:
balanced_df = pd.concat([df_t_0, df_t_1_sub])

In [7]:
balanced_df

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_210,feature_211,feature_212,feature_213,feature_214,feature_215,feature_216,feature_217,feature_218,target
22,0.661364,1.0,0.0,0.000120,0.310606,0.009740,0.180457,0.752510,0.001050,0.500000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.0,0
23,0.672847,0.5,0.0,0.000131,0.327720,0.035065,0.685714,0.777475,0.003627,0.000000,...,0.054111,0.097624,0.005062,0.86365,0.033398,0.156841,0.025219,0.002243,0.0,0
36,0.728351,0.5,0.0,0.000068,0.245778,0.001299,0.571429,0.712884,0.000604,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.0,0
67,0.575738,0.5,0.0,0.000000,0.259794,0.011688,0.228571,0.624854,0.000986,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0
72,0.505637,0.0,0.0,NaN,0.332217,0.001948,0.380914,0.847067,0.001845,0.666667,...,0.055426,0.058180,0.001687,0.75000,0.037872,0.134145,0.020597,0.003380,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429373,0.580473,0.5,0.0,0.000000,0.181268,0.001299,0.228571,0.774486,0.000986,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.0,1
465382,0.481473,1.0,0.0,0.000099,0.364699,0.005195,0.142857,0.753926,0.001782,0.750000,...,0.038695,0.377841,0.001125,0.75000,0.036198,0.105289,0.377841,0.004434,1.0,1
271088,0.644856,0.5,0.0,0.000000,0.366731,0.034416,0.175771,0.579792,0.001145,0.666667,...,0.142921,0.149654,0.001687,0.83335,0.068140,0.179926,0.020597,0.002200,1.0,1
269843,0.495677,1.0,0.0,NaN,0.151923,0.001948,0.285714,0.703029,0.000636,0.666667,...,0.024995,0.085543,0.001687,1.00000,0.025578,0.094812,0.085543,0.002970,1.0,1


In [8]:
balanced_df["target"].value_counts()

target
1    96466
0    66466
Name: count, dtype: int64

In [9]:
balanced_df = balanced_df.drop(["feature_15", "feature_19", "feature_39", "feature_148"], axis=1)

In [10]:
X = balanced_df.drop("target", axis=1)
y = balanced_df["target"].copy()

In [11]:
category_columns = ["feature_17", "feature_87", "feature_118", "feature_119", "feature_139", "feature_144", "feature_147", "feature_158", "feature_159", "feature_195"]

X_category = X[category_columns].copy()
X_numeric = X.drop(category_columns, axis=1)

In [12]:
EXPORT_PATH = "../../exports/"

with open(EXPORT_PATH+"category_label_encoders.pkl", 'rb') as file:
    category_label_encoders = pickle.load(file)

with open(EXPORT_PATH+"category_imputer.pkl", 'rb') as file:
    category_imputer = pickle.load(file)

with open(EXPORT_PATH+"numeric_imputer.pkl", 'rb') as file:
    numeric_imputer = pickle.load(file)

with open(EXPORT_PATH+"scaler.pkl", 'rb') as file:
    scaler = pickle.load(file)

In [13]:
preprocessed_data = preprocess_data(X_numeric=X_numeric, X_category=X_category, numeric_transformer=numeric_imputer, category_transformer=category_imputer, label_encoders=category_label_encoders, index=y.index)

In [14]:
preprocessed_data.isna().sum()

feature_1      0
feature_2      0
feature_3      0
feature_4      0
feature_5      0
              ..
feature_144    0
feature_147    0
feature_158    0
feature_159    0
feature_195    0
Length: 214, dtype: int64

In [15]:
#Scale data
scaled_data = scaler.transform(preprocessed_data.copy())
scaled_data = pd.DataFrame(scaled_data, columns=preprocessed_data.columns, index=y.index)

In [16]:
data = pd.concat([scaled_data, y], axis=1)

In [17]:
data.isna().sum()

feature_1      0
feature_2      0
feature_3      0
feature_4      0
feature_5      0
              ..
feature_147    0
feature_158    0
feature_159    0
feature_195    0
target         0
Length: 215, dtype: int64

In [18]:
data

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_87,feature_118,feature_119,feature_139,feature_144,feature_147,feature_158,feature_159,feature_195,target
22,0.972384,1.0,0.0,0.534527,0.116966,0.208333,-2.762467e-01,0.034810,-0.054054,0.000000,...,1.0,0.0,-0.25,-0.833333,0.0,0.0,0.50,0.6,2.0,0
23,1.050953,0.0,0.0,0.618926,0.229877,1.833333,2.624672e+00,0.202340,2.135135,-1.500000,...,-1.0,-1.0,-0.25,-0.833333,-1.0,-1.0,0.75,-0.4,0.0,0
36,1.430725,0.0,0.0,0.144501,-0.310734,-0.333333,1.968504e+00,-0.231107,-0.432432,-1.500000,...,0.0,0.0,0.00,-0.166667,0.0,0.5,0.75,0.6,2.0,0
67,0.386520,0.0,0.0,-0.373402,-0.218263,0.333333,3.187156e-16,-0.821849,-0.108108,-1.500000,...,0.0,0.0,0.00,-0.666667,-0.5,1.0,-0.75,-0.6,3.0,0
72,-0.093120,-1.0,0.0,0.000000,0.259550,-0.291667,8.746719e-01,0.669351,0.621622,0.500000,...,0.0,0.0,1.00,0.333333,0.0,-0.5,-0.50,-1.2,-1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429373,0.418921,0.0,0.0,-0.373402,-0.736338,-0.333333,3.187156e-16,0.182283,-0.108108,-1.500000,...,1.0,-1.0,1.00,0.333333,0.0,-1.0,0.25,-0.2,4.0,1
465382,-0.258455,1.0,0.0,0.374680,0.473851,-0.083333,-4.921261e-01,0.044310,0.567567,0.750000,...,1.0,1.0,1.00,0.333333,-1.0,-1.0,-0.75,-0.4,0.0,1
271088,0.859436,0.0,0.0,-0.373402,0.487252,1.791667,-3.031496e-01,-1.124245,0.027027,0.500000,...,1.0,-1.0,0.00,0.166667,0.0,-1.0,-0.75,-0.2,-1.0,1
269843,-0.161271,1.0,0.0,0.000000,-0.929941,-0.291667,3.280839e-01,-0.297242,-0.405405,0.500000,...,0.0,-1.0,0.50,-0.166667,0.0,-1.0,0.25,0.0,0.0,1


In [19]:
data.to_csv("../../data/processed/final/scaled_full_data.csv", index=False)